# day 3 tutorial

**users will need to install ipywidgets and ipyleaflet to use the widgets outside of MyBinder environment. Then they need to enable the widget extension for jupyter:**
```
pip install ipywidgets
pip install ipyleaflet
jupyter nbextension enable --py widgetsnbextension
```
**maybe we don't display the hovers. it's kinda hack-y implementation. ipyleaflet map widget doesn't have great options for this type of interaction. this is the purged hover handler ocde:**
```
def hover_handler(event=None, id=None, properties=None):
    if len(m.layers)==3:
        m.remove_layer(m.layers[-1])

    nyears = len([key for key in properties.keys() if "MEAN" in key])
    data = {'year': [], 'mean': [], 'std': []}
    for i in range(1,nyears):
        data['year'].append(1980+i)
        data['mean'].append(properties["MEAN_"+str(i)])
        data['std'].append(properties["STD_"+str(i)])
    df = pd.DataFrame(data)
        
    popup = mwg.Popup(location=properties['centroid'], 
                      child=wg.HTML("{put some info here}"), 
                      auto_close=True,
                      class_name="custom")
    m.add_layer(popup)
    
layer.on_hover(hover_handler)
m.add_layer(layer)
m
```
**purged imports:**
```
import math
import matplotlib as mpl
import matplotlib.pyplot as plt, mpld3
mpl.rcParams['figure.figsize'] = [8, 7]
```

In [1]:
# core
import os
import json
#import requests # not used yet; for downloads <<--
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# widgets/map colors
import ipywidgets as wg
import ipyleaflet as mwg
from matplotlib import cm, colors

# miscellaneous settings
smap_res = 9
basemap = mwg.basemap_to_tiles(mwg.basemaps.Esri.WorldImagery)

## open shapefile with geopandas and organize; get some basic spatial metadata

In [2]:
# read shapefile to geodataframe
gdf = gpd.GeoDataFrame.from_file("sites/Sites_lf.shp")  

# get geodataframe as wgs84l get centroid of each polygon
gdf = gdf.to_crs({'init': 'epsg:4326'})
gdf['centroid'] = [(p.y, p.x) for p in gdf.centroid]

# make list of colors using matplotlib's colormapper
gdf['style'] = [{
    'color': colors.rgb2hex(d[0:3]), 
    'fillColor': colors.rgb2hex(d[0:3]), 
    'weight': 1, 'fillOpacity': 0.5
} for d in cm.Set3(np.linspace(0.0,1.0,len(gdf)))]

# get geometries from geodataframe as python dictionary; get bounds
geodict = json.loads(gdf.to_json())
bnds, series = gdf.bounds, gdf['geometry']

# get extent and centroid for whole file
minx, maxx = min(bnds['minx']), max(bnds['maxx'])
miny, maxy = min(bnds['miny']), max(bnds['maxy'])
centroid = ((maxy+miny)/2, (maxx+minx)/2)

# display first two rows of geodataframe
gdf.head(2)

,OBJECTID,RANGERDIST,REGION,FORESTNUMB,DISTRICTNU,DISTRICTOR,FORESTNAME,DISTRICTNA,GIS_ACRES,SHAPE_Leng,...,STD_29,STD_30,STD_31,STD_32,STD_33,STD_34,STD_35,geometry,centroid,style
0,61,99030501010343,03,05,01,030501,Coronado National Forest,Douglas Ranger District,434025.200,3.963602,...,446.146563,453.982071,557.405375,339.841942,599.562885,470.501168,343.082608,(POLYGON ((-109.2462605499999 32.0543309500000...,"(31.774734099094083, -109.3275890439515)","{'color': '#8dd3c7', 'fillOpacity': 0.5, 'weig..."
1,62,99030503010343,03,05,03,030503,Coronado National Forest,Sierra Vista Ranger District,321534.997,2.854066,...,502.615547,430.908201,701.371540,272.786808,445.029396,525.713796,242.986111,(POLYGON ((-110.6378639499999 31.6445072200000...,"(31.505073066518996, -110.53696719971326)","{'color': '#ffffb3', 'fillOpacity': 0.5, 'weig..."


# EASE Grid

two binary files with (almost) global lat, lon coordinates:

In [3]:
lats = "docs/EASE2_M09km.lats.3856x1624x1.double"
lons = "docs/EASE2_M09km.lons.3856x1624x1.double"

# read the files to two numpy arrays
lat_array = np.fromfile(lats, dtype=np.float64).flatten()
lon_array = np.fromfile(lons, dtype=np.float64).flatten()

# zip the arrays with np.dstack
c = np.dstack((lon_array, lat_array))[0]
print("Array shape: "+str(c.shape))
c

Array shape: (6262144, 2)


array([[-179.9533195 ,   84.6564188 ],
       [-179.85995851,   84.6564188 ],
       [-179.76659751,   84.6564188 ],
       ...,
       [ 179.76659751,  -84.6564188 ],
       [ 179.85995851,  -84.6564188 ],
       [ 179.9533195 ,  -84.6564188 ]])

### Most straightforward way to select coordinates inside a polygon is to first index the coordinate array using the polygon extent:

In [4]:
selection = c[(-90<lon_array) & (lon_array<-85) & (25<lat_array) & (lat_array<30)]
selection

array([[-89.9533195 ,  29.94568162],
       [-89.85995851,  29.94568162],
       [-89.76659751,  29.94568162],
       ...,
       [-85.19190871,  25.02339698],
       [-85.09854772,  25.02339698],
       [-85.00518672,  25.02339698]])

### Simple function uses the logic above to select an array of coordinates within bounding extent of the polygon:

In [5]:
def get_ease(geom):
    
    # geom.bounds method returns a tuple of bounding coordinates:
    sel_minlon, sel_minlat, sel_maxlon, sel_maxlat = geom.bounds
    
    # index the xy array with bool conditions based on extent
    sel_ease = c[(sel_minlon<lon_array) & (lon_array<sel_maxlon) & 
                 (sel_minlat<lat_array) & (lat_array<sel_maxlat)]
    
    return(sel_ease)

### Test with polygon feature number 8, printing the first five coordinates:

In [6]:
get_ease(gdf.iloc[8].geometry)[:5]

array([[-107.50518672,   37.91860119],
       [-107.41182573,   37.91860119],
       [-107.31846473,   37.91860119],
       [-107.22510373,   37.91860119],
       [-107.13174274,   37.91860119]])

## initialize the map and add the site polygons

In [7]:
# initialize widgets
m = mwg.Map(layers=(basemap,), center=(33, -109), zoom=6, scroll_wheel_zoom=True) 
layer = mwg.GeoJSON(
    data=geodict, hover_style={'color': "white", 'weight': 2, 'fillOpacity': 0.5})

m.add_layer(layer)

## add click handler; display map
click handler calls `get_ease` upon polygon click, reduces array of ease grid centroids using polygon extent, uses shapely `contains()` to determine if point is inside polygon (geopandas is basically pandas with shapely geometry column), makes feature and adds to layer group.

In [8]:
def click_handler(**kwargs):
    
    # click callback returns two separate objects; differentiate
    if "id" not in kwargs.keys():
        return(None)
    
    # get geopandas row corresponding to clicked polygon; get the geometry
    g = gdf.iloc[int(kwargs['id'])]
    geom = g.geometry
    
    # use our function to select an array of coords within geom extent
    ease_grid = get_ease(geom)
    
    # iterate over array of coordinates; append contained points as layers
    point_features = []
    for pt in ease_grid:
        
        # if coordinates "pt" are inside the selected geometry,
        if geom.contains(Point(pt[0], pt[1])):
            
            # make a circle marker and append to list
            point_features.append(mwg.CircleMarker(location=(pt[1], pt[0])))

    # make a layer group from the list of point features
    point_group = mwg.LayerGroup(layers=tuple(point_features))
  
    # get the centroid of selected layer; center and zoom the map
    centroid = geom.centroid
    m.center = (centroid.y, centroid.x)
    m.zoom = 9
    
    # finally, add all of the layers
    m.add_layer(point_group)
        
layer.on_click(click_handler)
m

Map(basemap={'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a…